In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/11 22:40:56 WARN Utils: Your hostname, kirans-mac.local, resolves to a loopback address: 127.0.0.1; using 172.18.197.149 instead (on interface en0)
25/06/11 22:40:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 22:40:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/11 22:40:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Broad cast variables

In [4]:
#Broadcast a Dictionary
days={"sun": "Sunday", "mon" : "Monday", "tue":"Tuesday"}
bcDays = spark.sparkContext.broadcast(days)
bcDays.value
bcDays.value['sun']

'Sunday'

In [6]:
type(bcDays)

pyspark.core.broadcast.Broadcast

In [7]:
numbers = (1,2,3)
broadcastNumbers=spark.sparkContext.broadcast(numbers)
broadcastNumbers.value
broadcastNumbers.value[0]

1

In [8]:
type(broadcastNumbers)

pyspark.core.broadcast.Broadcast

### real problem to use broadcast variables -> convert mon to monday, etc

In [11]:
data = (("James","Smith","USA","mon"),
("Michael","Rose","USA","tue"),
("Robert","Williams","USA","sun"),
("Maria","Jones","USA","tue")
)

rdd = spark.sparkContext.parallelize(data)
rdd.take(5)

[('James', 'Smith', 'USA', 'mon'),
 ('Michael', 'Rose', 'USA', 'tue'),
 ('Robert', 'Williams', 'USA', 'sun'),
 ('Maria', 'Jones', 'USA', 'tue')]

In [24]:
days={"sun": "Sunday", "mon" : "Monday", "tue":"Tuesday"}
bcDays = spark.sparkContext.broadcast(days)

def days_convert(day):
    return bcDays.value.get(day, "Unknown") 

rdd.map(lambda x: (x[0],x[1],x[2],days_convert(x[3]))).collect()

[('James', 'Smith', 'USA', 'Monday'),
 ('Michael', 'Rose', 'USA', 'Tuesday'),
 ('Robert', 'Williams', 'USA', 'Sunday'),
 ('Maria', 'Jones', 'USA', 'Tuesday')]

# Accumulator variables

In [25]:
# Create an accumulator
blankLines = spark.sparkContext.accumulator(0)

# Sample data
data = ["Hello", "", "World", "Spark", "", "Accumulator", ""]

# Parallelize data into RDD
rdd = spark.sparkContext.parallelize(data)

# Function to check and count blank lines
def countBlank(line):
    global blankLines
    if line == "":
        blankLines += 1
    return line

# Apply the function to the RDD
rdd.foreach(countBlank)

# Get the result
print("Number of blank lines:", blankLines.value)

Number of blank lines: 3


In [26]:
type(blankLines)

pyspark.accumulators.Accumulator